In [1]:
import sqlite3 as sql
import pandas as pd

In [2]:
conn = sql.connect('pokedex.db')
c = conn.cursor()

def executeScriptsFromFile(filename):
    # Open and read the file as a single buffer
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()

    # all SQL commands (split on ';')
    sqlCommands = sqlFile.split(';')

    # Execute every command from the input file
    for command in sqlCommands:
        c.execute(command)

In [ ]:
executeScriptsFromFile('sql/criacao_tabelas.sql')

executeScriptsFromFile('sql/adc_pokemon.sql')
executeScriptsFromFile('sql/adc_tipos_pokemon.sql')
executeScriptsFromFile('sql/adc_ataques.sql')
executeScriptsFromFile('sql/adc_uso_ataque.sql')
executeScriptsFromFile('sql/adc_habilidades.sql')
executeScriptsFromFile('sql/adc_uso_habilidades.sql')
executeScriptsFromFile('sql/adc_uso_pokemon.sql')
executeScriptsFromFile('sql/add_evolucoes.sql')
executeScriptsFromFile('sql/adc_aliados.sql')

In [ ]:
# Consulta 1.1 (Operações de Seleção e Projeção simples)
# Lista os conteúdos da tabela Pokémon (Nome do pokemon, junto com
# sua descrição e número)
consulta1_1 = pd.read_sql_query("SELECT *"
                                + "FROM Pokemon", conn)
consulta1_1

In [ ]:
# Consulta 1.2 (Operações de Seleção e Projeção simples)
# Lista os conteúdos da tabela Ataques (Nomes dos ataques, junto
# da sua descrição, tipo e número de identificação)
consulta1_2 = pd.read_sql_query("SELECT *"
                                + "FROM Ataque", conn)
consulta1_2

In [ ]:
# Consulta 2.1 (Junção de 2 Relações)
# Lista os Pokémons que são do tipo fogo e agua simultaneamente
consulta2_1 = pd.read_sql_query("SELECT nome"
                                + " FROM Pokemon NATURAL JOIN TipoPokemon"
                                + " WHERE tipo = 'fogo' AND nome IN(SELECT nome"
                                + " FROM Pokemon NATURAL JOIN TipoPokemon"
                                + " WHERE tipo = 'agua')", conn)
consulta2_1

In [ ]:
# Consulta 2.2 (Junção de 2 Relações)
# Lista os Pokémons que não possuem evolução
# e que não são evolução de outro pokemon.
# A junção de 2 Relações se encontra no 2º SELECT 
consulta2_2 = pd.read_sql_query("SELECT nome"
                                + " FROM Pokemon"
                                + " WHERE nome NOT IN(SELECT DISTINCT nome"
                                + " FROM Pokemon JOIN Evolucao ON id_de = id_pokemon OR id_para = id_pokemon)"
                                , conn)
consulta2_2

In [ ]:
# Consulta 2.3 (Junção de 2 Relações)
# Lista os Pokémons com uso maior ou igual a 25%
# mostrando seu nome, o ano, mes e a porcentagem
# com os anos em ordem decrescente e os meses de cada ano de forma crescente
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
consulta2_3 = pd.read_sql_query("SELECT nome, ano, mes, porcentagem"
                                + " FROM Pokemon NATURAL JOIN UsoPokemon"
                                + " WHERE porcentagem >= 0.25"
                                + " ORDER BY ano DESC, mes ASC", conn)
consulta2_3

In [ ]:
# Consulta 3.1 (Junção de 3 ou mais Relações)
# Lista de Habilidades por Pokemon, em ordem de uso (mais usadas às menos usadas)
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
consulta3_1 = pd.read_sql_query('SELECT p.nome, h.nome, h.descricao'
                                + ' FROM (Habilidade AS h NATURAL JOIN UsoHabilidade AS uh)'
                                + ' JOIN Pokemon AS p ON p.id_pokemon = uh.id_pokemon'
                                + ' GROUP BY p.id_pokemon, h.id_habilidade'
                                + ' ORDER BY p.id_pokemon ASC, uh.porcentagem DESC', conn)
consulta3_1

In [ ]:
# Consulta 3.2 (Junção de 3 ou mais Relações)
# Lista de Ataques por Pokemon, em ordem de uso (mais usadas às menos usadas)
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
consulta3_2 = pd.read_sql_query('SELECT p.nome, a.nome, a.descricao'
                                + ' FROM (Ataque AS a NATURAL JOIN UsoAtaque AS ua)'
                                + ' JOIN Pokemon AS p ON p.id_pokemon = ua.id_pokemon'
                                + ' GROUP BY p.id_pokemon, a.id_ataque'
                                + ' ORDER BY p.id_pokemon ASC, ua.porcentagem DESC', conn)
consulta3_2

In [ ]:
# Consulta 3.3 (Junção de 3 ou mais Relações)
# Lista de Pokemons que, em um dado mês e ano, já foram mais utilizados em batalha
# do que suas evoluções, e qual evolução à que se refere
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
consulta3_3 = pd.read_sql_query('SELECT DISTINCT p1.nome, p2.nome'
                                + ' FROM (((Pokemon NATURAL JOIN UsoPokemon) AS p1 JOIN Evolucao as ev'
                                + ' ON p1.id_pokemon = ev.id_de) JOIN (Pokemon NATURAL JOIN UsoPokemon) as p2'
                                + ' ON p2.id_pokemon = ev.id_para)'
                                + ' WHERE p1.ano = p2.ano AND p1.mes = p2.mes'
                                + ' AND p1.porcentagem > p2.porcentagem', conn)
consulta3_3

In [ ]:
# Consulta 4.1 (Funções de Agregação):
# Retorna o Pokemon mais utilizado em batalhas durante todo o tempo de 2014 a 2019
# (do nível OU de batalha da Smogon)
consulta4_1 = pd.read_sql_query('SELECT nome'
                         + ' FROM Pokemon NATURAL JOIN UsoPokemon'
                         + ' WHERE porcentagem = '
                         + '(SELECT MAX(porcentagem) FROM UsoPokemon)', conn)
consulta4_1

In [ ]:
# Consulta 4.1 (Funções de Agregação):
# Retorna uma lista com os anos e os tipos utilizados em batalhas,
# agrupados por ano (decrescente) por ordem de uso (do mais utilizado ao menos utilizado daquele ano)
# (do nível OU de batalha da Smogon)
consulta4_2 = pd.read_sql_query('SELECT ano, tipo'
                                + ' FROM UsoPokemon NATURAL JOIN Pokemon NATURAL JOIN TipoPokemon'
                                + ' GROUP BY ano, tipo'
                                + ' ORDER BY ano DESC, AVG(porcentagem) DESC', conn)
consulta4_2

In [ ]:
# Relatório 1: Porcentagem de uso de habilidade por Pokémon
# no mês de dezembro de 2018
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
relatorio_1 = pd.read_sql_query("SELECT p.nome AS pokemon, h.nome AS habilidade, uh.porcentagem"
                                + " FROM (Habilidade AS h NATURAL JOIN UsoHabilidade AS uh)"
                                + " JOIN Pokemon AS p ON p.id_pokemon = uh.id_pokemon"
                                + " WHERE ano = 2018 AND mes = 12"
                                + " ORDER BY p.id_pokemon ASC", conn)
relatorio_1

In [ ]:
# Relatório 2: Porcentagem de uso de Ataque por Pokémon
# no mês de agosto de 2015
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
relatorio_2 = pd.read_sql_query('SELECT p.nome AS pokemon, a.nome AS ataque, ua.porcentagem'
                                + ' FROM (Ataque AS a NATURAL JOIN UsoAtaque AS ua)'
                                + ' JOIN Pokemon AS p ON p.id_pokemon = ua.id_pokemon'
                                + ' WHERE ano = 2015 AND mes = 8'
                                + ' ORDER BY p.id_pokemon ASC', conn)
relatorio_2

In [ ]:
# Relatório 3: Lista de Aliados Comuns por Pokemon
# no mês de abril de 2017
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
relatorio_3 = pd.read_sql_query("SELECT p1.nome AS pokemon, p2.nome AS aliado, a.porcentagem"
                                + " FROM (Pokemon AS p1 JOIN AliadosComuns AS a"
                                + " ON p1.id_pokemon = a.id_pokemon) JOIN Pokemon AS p2"
                                + " ON p2.id_pokemon = a.id_aliado"
                                + " WHERE ano = 2017 AND mes = 4"
                                + " ORDER BY p1.id_pokemon ASC", conn) 
relatorio_3